## Remove the extra files

This script assumes that you have copied the BasicGuardDutyWorking directory into
the BasicGuardDutyWorkingPhaseTwo directory.

This script removes `_events.tsv` and `_events_temp.tsv` and then
renames `_events_temp1.tsv` to `_events.tsv`.

In [ ]:
import os
from hed.tools import BidsTsvDictionary, HedLogger
from hed.util import get_file_list

# Variables to set for the specific dataset
bids_root_path = 'F:/ARLBidsStart/BasicGuardDutyWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'ses', 'run')
log_file_name = 'bcit_basic_guard_duty_03_remove_files_log.json'

# Construct the event file dictionaries for the BIDS and for EEG.event files
files_bids = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events", exclude_dirs=exclude_dirs)
bids_dict = BidsTsvDictionary("Bids event files", files_bids, entities=entities)

# Set logger and remove the appropriate files
logger = HedLogger()

for key, file in bids_dict.iter_files():
    # Remove events.tsv
    filename = file.file_path
    os.remove(filename)
    logger.add(key, f"Remove {filename}")

    # Remove events_temp.tsv
    filename_temp = filename[:-4] + "_temp.tsv"
    if os.path.isfile(filename_temp):
        os.remove(filename_temp)
        logger.add(key, f"Removed {filename_temp}")
    else:
        logger.add(key, f"{filename_temp} does not exist and could not be removed", level="WARNING", also_print=True)

    # Move events_temp1.tsv to events.tsv
    filename_temp1 = filename[:-4] + "_temp1.tsv"
    if os.path.isfile(filename_temp1):
        os.rename(filename_temp1, filename)
        logger.add(key, f"Renamed {filename_temp1}\n\t\tto {filename}")
    else:
        logger.add(key, f"{filename_temp1} does not exist and could not be moved", level="WARNING", also_print=True)

# Now make sure there aren't any unexpected leftovers
temp_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events_temp",
                           exclude_dirs=exclude_dirs)
if temp_files:
    for file in temp_files:
        logger.add('', f"{file} is unexpectedly left over", level="WARNING", also_print=True)

temp1_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events_temp1",
                           exclude_dirs=exclude_dirs)
if temp1_files:
    for file in temp1_files:
        logger.add('', f"{file} is unexpectedly left over", level="WARNING", also_print=True)

# Output and save the log
logger.print_log()
save_path = os.path.join(bids_root_path, 'code')
logger.save_log(save_path, log_name=log_file_name)

# Output errors
print("\nWARNING Summary:")
logger.print_log(level="WARNING")